## ML Assignment 2
### Progamming Part

### Implementing KNN using intersection Distance and Jaccard Distance
### Submitted By:- Ayush Sethi (as11500) , Darshil Patel (ddp337)

In [132]:
import pandas as pd

def inter(c1, c2):
    distance = 0
    token1 = c1.split()
    token2 = c2.split()

    token1 = distinct(token1)
    token2 = distinct(token2)
    for tok in token1:
        if tok in token2:
            distance = distance +1
    
    if distance == 0:
        return 1000
    
    return 1/distance

def distinct(token1):
    dist_token = []
    for i in token1:
        if i not in dist_token:
            dist_token.append(i)
    return dist_token

def majority(a):
    ones = 0
    two = 0
    for i in a:
        if i == '1':
            ones = ones + 1
        else:
            two = two + 1
    if ones == two:
        ones = ones +1
            
    return 1 if ones>two else 0

def confusion_matrix(true, pred):
    true_positive = 0
    false_negative = 0
    false_positive = 0
    true_negative = 0
    for i,j in zip(true,pred):
        if int(i) == 1 and int(i) == j:
            true_positive = true_positive + 1
        if int(i) == 0 and int(i) == j:
            true_negative = true_negative + 1
        if int(i) == 1 and int(i) != j:
            false_negative = false_negative + 1
        if int(i) == 0 and int(i) != j:
            false_positive = false_positive + 1
    
    mat = [[true_positive,false_negative],[false_positive,true_negative]]
    accuracy = (true_positive + true_negative)/(true_positive + false_negative + false_positive + true_negative)
    print(accuracy)
    return mat, accuracy

def jaccard(a, b):
    c = a & b
    return 1-float(len(c)) / (len(a) + len(b) - len(c))

def myFunc1(test_comment, train_comment):
    docA = test_comment
    docB = train_comment
    bowA = set(docA.split())
    bowB = set(docB.split())        
    return jaccard(bowA, bowB)

def knn(test_comment,k,train,func):
    dist_vector = [(func(test_comment, row.comment), row.comment, row.labels, row.idx) for index, row in train.iterrows()]
    dist_vector = pd.DataFrame(dist_vector,columns = ['dist','comment','labels','idx'])
    best_dist = dist_vector.nsmallest(k,'dist').dist
    
    best_dist = best_dist.tolist()
    best_labels = [vector[2] for index, vector in dist_vector.iterrows() if vector[0] in best_dist]
    return majority(best_labels)


def cross_validation(full_set,folds, k, func):
    avg = len(full_set) / float(folds)
    out = []
    last = 0.0
    CV_accuracy = 0
#     correct_prediction = 0
#     total_prediction = 0

    while last < len(full_set):
        out.append(full_set[int(last):int(last + avg)])
        last += avg

    #print(out)
    for fold in range(len(out)):
        print("trying fold:",fold)
        training  = pd.DataFrame()
        frames = [training]
        
        for i in range(len(out)):
            if i!= fold:
                frames.append(out[i])
                
        training = pd.concat(frames)                
        prediction = []
        for i in range(len(out[fold])):
            prediction.append(knn(out[fold].iloc[i].comment, k, training, func))
        
        matrix, accuracy = confusion_matrix(out[fold].labels, prediction)
#         correct_prediction = correct_prediction + matrix[0][0] + matrix[1][1]
#         total_prediction = total_prediction + matrix[0][0] + matrix[0][1] + matrix[1][0] + matrix[1][1]
        CV_accuracy = CV_accuracy + accuracy
     
    return CV_accuracy

In [133]:
train = pd.read_csv('reviewstrain.txt', sep = "/n", header = None, engine = 'python')
train['labels'] = train[0].str[0:1]
train['comment'] = train[0].str[1:]
train['idx'] = train.index
train.drop(0,axis=1,inplace=True)
#train_token = [list(set(row.comment.split())) for index, row in train.iterrows()]

test = pd.read_csv('reviewstest.txt', sep = "/n", header = None, engine = 'python')
test['labels'] = test[0].str[0:1]
test['comment'] = test[0].str[1:]
test['idx'] = test.index
test.drop(0,axis=1,inplace=True)
#test_token = [list(set(row.comment.split())) for index, row in test.iterrows()]

## 1) For k = 1, what is the predicted label for the following example in the test set: It
## leaves little doubt that Kidman has become one of our best actors .

In [134]:
knn(test.iloc[17].comment, 1, train, inter)

1

## 2&3 What is the confusion matrix (on the test set) for k = 1?
## iii. Report the accuracy, the true positive rate, and the false positive rate, on the test
## set for k = 1.

In [135]:
prediction = []
for i in range(len(test)):
    prediction.append(knn(test.iloc[i].comment,1, train,inter))

matrix, accuracy = confusion_matrix(test.labels,prediction)
print("Confusion Matrix", matrix)
print("Accuracy:", accuracy)
tpr = matrix[0][0]/(matrix[0][0] + matrix[0][1])
print("TRUE POSITIVE RATE", tpr)
fpr = matrix[1][0]/(matrix[1][0] + matrix[1][1])
print("FALSE POSITIVE RATE", fpr)

0.604
Confusion Matrix [[209, 64], [134, 93]]
Accuracy: 0.604
TRUE POSITIVE RATE 0.7655677655677655
FALSE POSITIVE RATE 0.5903083700440529


##  For k = 5, what is the predicted label for the following example in the test set: It
## leaves little doubt that Kidman has become one of our best actors .


In [136]:
knn(test.iloc[17].comment,5, train, inter)

1

## What is the confusion matrix (on the test set) for k = 5?
## vi. Report the accuracy, the true positive rate, and the false positive rate, on the test set for k = 5.
## vii. What is the accuracy on the test set for k = 5?

In [137]:
prediction = []
for i in range(len(test)):
    prediction.append(knn(test.iloc[i].comment,5, train, inter))

matrix, accuracy = confusion_matrix(test.labels,prediction)
print("Confusion Matrix", matrix)
print("Accuracy:", accuracy)
tpr = matrix[0][0]/(matrix[0][0] + matrix[0][1])
print("TRUE POSITIVE RATE", tpr)
fpr = matrix[1][0]/(matrix[1][0] + matrix[1][1])
print("FALSE POSITIVE RATE", fpr)

0.606
Confusion Matrix [[212, 61], [136, 91]]
Accuracy: 0.606
TRUE POSITIVE RATE 0.7765567765567766
FALSE POSITIVE RATE 0.5991189427312775


## viii. Suppose we used the very simple Zero-R classifier

In [138]:
# Zero R classifier
# 1 is most common
df_filtered = train[(train.labels == '1')]
len(df_filtered)

prediction = []
for i in range(len(test)):
    prediction.append(knn(test.iloc[i].comment, 5, train, inter))

print(confusion_matrix(test.labels,[1]*500))

0.546
([[273, 0], [227, 0]], 0.546)


### Search for best K

In [140]:
list_K = [3, 7, 99]
def findBestK(list_K):
    K_accuracy = dict()
    for i in list_K:
        K_accuracy[i] = cross_validation(train, 5, i, inter)
    
    #print(K_accuracy)    
    return min(sorted(K_accuracy.items(), key=lambda x: x[1]))  
        
    
    
    
print(findBestK(list_K)) 

best_K = 3
# 3 Has the highset Accuracy
# {3: 0.6599999999999999, 7: 0.658, 99: 0.612}

trying fold: 0
0.6833333333333333
trying fold: 1
0.6766666666666666
trying fold: 2
0.6633333333333333
trying fold: 3
0.63
trying fold: 4
0.6466666666666666
trying fold: 0
0.65
trying fold: 1
0.6733333333333333
trying fold: 2
0.71
trying fold: 3
0.6533333333333333
trying fold: 4
0.6033333333333334
trying fold: 0
0.6366666666666667
trying fold: 1
0.5966666666666667
trying fold: 2
0.6633333333333333
trying fold: 3
0.6066666666666667
trying fold: 4
0.5566666666666666
(3, 3.3)


## Accuracy on highest k using test set

In [141]:
prediction = []
for i in range(len(test)):
    prediction.append(knn(test.iloc[i].comment,best_K, train, inter))

matrix, accuracy = confusion_matrix(test.labels,prediction)
print("Confusion Matrix", matrix)
print("Accuracy:", accuracy)

0.59
Confusion Matrix [[212, 61], [144, 83]]
Accuracy: 0.59


## With New distance function

## k = 1

In [127]:
prediction = []
for i in range(len(test)):
    prediction.append(knn(test.iloc[i].comment,1, train, myFunc1))

matrix, accuracy = confusion_matrix(test.labels,prediction)
print("Confusion Matrix", matrix)
print("Confusion Matrix", accuracy)
tpr = matrix[0][0]/(matrix[0][0] + matrix[0][1])
print("TRUE POSITIVE RATE", tpr)
fpr = matrix[1][0]/(matrix[1][0] + matrix[1][1])
print("FALSE POSITIVE RATE", fpr)

0.642
Confusion Matrix [[206, 67], [112, 115]]
Confusion Matrix 0.642


## k = 5

In [128]:
prediction = []
for i in range(len(test)):
    prediction.append(knn(test.iloc[i].comment,5, train, myFunc1))

matrix, accuracy = confusion_matrix(test.labels,prediction)
print("Confusion Matrix", matrix)
print("Confusion Matrix", accuracy)
tpr = matrix[0][0]/(matrix[0][0] + matrix[0][1])
print("TRUE POSITIVE RATE", tpr)
fpr = matrix[1][0]/(matrix[1][0] + matrix[1][1])
print("FALSE POSITIVE RATE", fpr)

0.668
Confusion Matrix [[220, 53], [113, 114]]
Confusion Matrix 0.668
TRUE POSITIVE RATE 0.8058608058608059
FALSE POSITIVE RATE 0.4977973568281938
